## **Boulder Grasslands Phenology Analysis**

## Load libraries

In [ ]:
# Load libraries
import cartopy.crs as ccrs
import earthpy as et
import earthpy.earthexplorer as etee
import earthpy.appeears as etapp
import pandas as pd
import geopandas as gpd
import geoviews as gv
from glob import glob
import holoviews as hv
import hvplot.pandas
import hvplot.xarray
import io
import numpy as np
import os
import requests
import regionmask
import rioxarray as rxr
import seaborn as sns
import xarray as xr

from shapely.geometry import box, Polygon

import matplotlib.pyplot as plt

hv.extension('bokeh')
gv.extension("bokeh")

import pathlib

## Download data

Make directories.

In [ ]:
# Make data directory

data_dir = os.path.join(et.io.HOME, et.io.DATA_NAME, 'boulder-grasslands')
ndvi_dir = os.path.join(data_dir, 'ndvi-data')
veg_type_dir = os.path.join(data_dir, 'veg-type-data')

ndvi_processed_data_path = os.path.join(ndvi_dir, 'processed_data')

for a_dir in [ndvi_dir, veg_type_dir, ndvi_processed_data_path]:
        if not os.path.exists(a_dir):
                os.makedirs(a_dir)

Download vegetation data.

In [ ]:
# Test: Download for one chunk
# veg_url = (
#     "https://gis.bouldercolorado.gov/ags_svr2/rest/services/osmp/OSMPVegetation/MapServer/1/query?where=OBJECTID%20%3E%3D%20{min_objectid}%20AND%20OBJECTID%20%3C%3D%20{max_objectid}&outFields=*&outSR=4326&f=geojson"
# )

# user_agent = (
#     'Mozilla/5.0 (X11; Linux x86_64; rv:60.0) '
#     'Gecko/20100101 Firefox/81.0'
# )
# r = requests.get(url=veg_url.format(min_objectid=11444, max_objectid=11447), headers={'User-Agent': user_agent})

# # Read GeoJSON data into a GeoDataFrame
# geojson_data = r.json()

# veg_gdf = gpd.GeoDataFrame.from_features(geojson_data['features'])

# veg_gdf

In [ ]:
# Download vegetation type data if it does not exist

print("Checking if data is downloaded...")
veg_type_path = os.path.join(veg_type_dir, 'veg_type.geojson')

if os.path.exists(veg_type_path):
    print("Data is already downloaded.")  
else: 
    print("Data is not downloaded. Initiating download...")

    # Define URL
    veg_url = (
        "https://gis.bouldercolorado.gov/ags_svr2/rest/services/osmp/OSMPVegetation/MapServer/1/query?where=1%3D1&outFields=*&returnGeometry=false&returnIdsOnly=true&outSR=4326&f=json"
    )

    # Mimic web browser
    user_agent = (
        'Mozilla/5.0 (X11; Linux x86_64; rv:60.0) '
        'Gecko/20100101 Firefox/81.0'
    )

    # Download GEOJSON
    r = requests.get(url=veg_url, headers={'User-Agent': user_agent})

    # Read GeoJSON data into a GeoDataFrame
    geojson_data = r.json()

    # Extract the objectIDs (the indexes of the rows in the dataset)
    objectid_list = geojson_data["objectIds"]

    # Define chunks
    chunks = [
        (objectid_list[i], 
            objectid_list[min(i + 1000,
                                len(objectid_list)-1)]) 
                                for i in range(0, len(objectid_list), 1000)
    ]
    print("Data chunks identified.")

    ## Download data in chunks

    # Create list
    veg_list = []

    # Due to the City of Boulder ArcGIS Hub limit of downloading
    # a maximum of 1,000 items at a time,
    # split the dataset into chunks and download the chunks individually.

    # Download data for each chunk
    for (min_objectid, max_objectid) in chunks:

        print("Downloading chunk.")

        # Define url
        veg_url = (
            "https://gis.bouldercolorado.gov/ags_svr2/rest/services/osmp/OSMPVegetation/MapServer/1/query?where=OBJECTID%20%3E%3D%20{min_objectid}%20AND%20OBJECTID%20%3C%3D%20{max_objectid}&outFields=*&outSR=4326&f=geojson"
        )

        # Mimic web browser
        user_agent = (
            'Mozilla/5.0 (X11; Linux x86_64; rv:60.0) '
            'Gecko/20100101 Firefox/81.0'
        )

        # Download chunk of data
        r = requests.get(url=veg_url.format(min_objectid=min_objectid, max_objectid=max_objectid), headers={'User-Agent': user_agent})

        # Read GeoJSON data into a GeoDataFrame
        geojson_data = r.json()
        veg_gdf = gpd.GeoDataFrame.from_features(geojson_data['features'])

        # Add the chunk gdf to the list
        veg_list.append(veg_gdf)

        print("Done.")

    # Concatenate the chunk gdfs into one gdf
    veg_gdf = pd.concat(veg_list)

    # Save downloaded data to CSV in directory
    veg_gdf.to_file(veg_type_path, driver='GeoJSON')
    print("Saved data to GeoJSON.")

Download NDVI data.

In [ ]:
# Load vegetation data
veg_gdf = gpd.read_file(veg_type_path)

# Calculate the total bounds
bounding_box = veg_gdf.total_bounds

# Create a polygon from the bounding box
minx, miny, maxx, maxy = bounding_box
bounding_box_polygon = box(minx, miny, maxx, maxy)
bounding_box_gdf = gpd.GeoDataFrame(geometry=[bounding_box_polygon],
                                    crs=veg_gdf.crs)

bounding_box_gdf

In [ ]:
# To test bbox code

# bbox_map = bounding_box_gdf.hvplot(
#     geo=True,
#     line_color='black',
#     fill_alpha=0,
#     tiles='EsriImagery'
# ).opts(
#     width=800,
#     height=800,
#     show_legend=False  # Set legend to False to remove it
# )

# bbox_map


In [ ]:
# Initialize AppeearsDownloader for MODIS NDVI data
ndvi_downloader = etapp.AppeearsDownloader(
    download_key="modis-ndvi",
    ea_dir=ndvi_dir,
    product="MYD13Q1.061",  # from list of APPEEARS datasts
    layer="_250m_16_days_NDVI",
    start_date="01-01",
    end_date="12-31",
    recurring=True,
    year_range=[2015, 2022],
    polygon=bounding_box_gdf,
)

# Download files if the download directory does not exist
if os.path.exists(ndvi_downloader.data_dir):
    print("MODIS NDVI data is already downloaded.")
else:
    print("Downloading MODIS NDVI data.")
    ndvi_downloader.download_files()

ndvi_downloader

Merge NDVI data into a Dataset.

In [ ]:
# Merge arrays and cache result

# Define NDVI processed data path
ndvi_combined_path = os.path.join(ndvi_processed_data_path, "ndvi_data.nc")


# Load and merge arrays only if the processed data has not already been created
if os.path.exists(ndvi_combined_path):
    print("NDVI data has already been merged and processed.")
else:
    print("Merging and processing data.")

    # Generate list of data files
    ndvi_path_list = glob(
        os.path.join(ndvi_downloader.data_dir, "*", "*NDVI*.tif")
    )

    # Merge images into a single data array

    doy_start = -19 # the character number of the start of doy in file name
    doy_end = -12 # the character number of the end of doy in file name
    scale_factor = 10000 # from MODIS data documentation

    # Define a list
    ndvi_da_list = []

    # For every file (.tif image), add it to the list
    for ndvi_path in ndvi_path_list:
        # Get date from file name
        doy = ndvi_path[doy_start:doy_end]

        # Define the date variable as the doy in file name
        date = pd.to_datetime(doy, format='%Y%j')

        # Open dataset
        da = rxr.open_rasterio(ndvi_path,
                            # masked=True changes specific excluded
                            # values from the metadata to NaN values
                            masked=True).squeeze()

        # Prepare to concatenate: Add date dimension and clean up metadata
        da = da.assign_coords({'date': date})
        da = da.expand_dims({'date': 1})
        da.name = 'NDVI'

        # Divide by scale factor
        da = da / scale_factor

        # Add the DataArray to the end of the accumulator list
        ndvi_da_list.append(da)
        print("Added .tif data to data array list.")

    # Stack arrays into time series
    ndvi_dataset = xr.combine_by_coords(ndvi_da_list, coords=["date"])
    print("Stacked arrays into data set.")

    # Cache the ndvi dataset as a netCDF
    ndvi_dataset.to_netcdf(path=ndvi_combined_path)
    print("Created netCDF file.")


## Load and explore data

In [ ]:
# View vegetation data

veg_gdf

In [ ]:
# Plot with base plotting

veg_gdf.plot()

In [ ]:
# Load NDVI data

ndvi_ds = xr.open_dataset(ndvi_combined_path)
ndvi_ds

NDVI [should take values between -1.0 and 1.0.](https://ipad.fas.usda.gov/cropexplorer/Definitions/spotveg.htm) Verify that NDVI falls within expected values.

In [ ]:
# Generate histogram of NDVI

ndvi_ds.NDVI.plot()

In [ ]:
# Plot NDVI in 2022 to test

# Select NDVI in 2022 June
ndvi_2022_da = (ndvi_ds
    .sel(date = '2022-06')
    .mean('date')
    .NDVI)

# Plot with matplotlib
ndvi_2022_da.plot(cmap=plt.colormaps['PiYG'])
veg_gdf.plot(facecolor='none', ax=plt.gca())

In [ ]:
# Visualize NDVI patterns over time
ndvi_time = gv.Dataset(ndvi_ds, kdims=['x', 'y', 'date'], vdims=['NDVI'])

# Create a GeoViews plot
ndvi_plot = ndvi_time.to(gv.Image).opts(cmap='greens', colorbar=True, alpha=0.8, tools=['hover'], width=700, height=500)
ndvi_ref_plot = ndvi_plot * gv.tile_sources.OSM()

ndvi_ref_plot

Now, only visualize NDVI within the vegetation polygon boundaries (i.e., ignore all NDVI pixels for which we will not calculate brown-down such as urban and agricultural lands).

In [ ]:
# Before clipping, check CRS
print(veg_gdf.crs)
print(ndvi_ds.rio.crs) 

In [ ]:
# Clip raster to polygons
ndvi_ds_veg_only = ndvi_ds.rio.clip(veg_gdf.geometry, all_touched=True)

print(ndvi_ds_veg_only.rio.crs)

all_touched option for the clip if that's what's causing the NAs

Can clip to debug.

In [ ]:
# Plot clipped data

# Calculate NDVI in 2022 June
ndvi_2022_clipped_da = (ndvi_ds_veg_only
    .sel(date = '2022-06')
    .mean('date')
    .NDVI)

# Plot with matplotlib
ndvi_2022_clipped_da.plot(cmap=plt.colormaps['PiYG'])
veg_gdf.plot(facecolor='none', ax=plt.gca())

In [ ]:
# Visualize NDVI over time
ndvi_time_veg_only = gv.Dataset(ndvi_ds_veg_only, kdims=['x', 'y', 'date'], vdims=['NDVI'])

# Create a GeoViews plot
ndvi_veg_only_plot = ndvi_time_veg_only.to(gv.Image).opts(cmap='viridis_r', colorbar=True, alpha=0.8, tools=['hover'], width=700, height=500)
ndvi_veg_only_plot * gv.tile_sources.EsriImagery()

## Calculate brown-down

In [ ]:
# Calculate percentile thresholds

thresholds_df = (ndvi_ds.to_dataframe()
                  .reset_index()
                  # Select only June-December dates - we are 
                  # not interested in green-up after snowmelt
                  .loc[lambda x: (x['date'].dt.month >= 6) & (x['date'].dt.month <= 12)]
                  # Convert the data coordinate to column
                  .assign(year=lambda x: x['date'].dt.year)
                  # Calculate 10th percentile value
                  .groupby(['x', 'y', 'year'])
                  .agg(threshold=('NDVI', lambda x: x.quantile(0.1)))

)
thresholds_df

In [ ]:
# Identify the 10th percentile dates of greenness

dates_da = (ndvi_ds_veg_only.to_dataframe()
            .reset_index()
            # Select only June-December dates - we are 
            # not interested in green-up after snowmelt
            .loc[lambda x: (x['date'].dt.month >= 6) & (x['date'].dt.month <= 12)]
            # Convert the data coordinate to column
            .assign(year=lambda x: x['date'].dt.year)
            # Left-join the thresholds by columns x, y, year
            .merge(thresholds_df, on=['x', 'y', 'year'], how='left')
            # Create a new column for TF the ndvi value falls at or under the threshold
            .assign(is_below_threshold=lambda x: x['NDVI'] <= x['threshold'])
            # Remove all False columns
            .loc[lambda x: x['is_below_threshold']]
            # Group by x, y, year, select lowest dates
            .groupby(['x', 'y', 'year'])
            # .agg(min_date=('date', np.nanmin))
            .agg(min_date=('date', 'min'))
            .to_xarray()

)

# Create day of year from date column
dates_da['day_of_year'] = dates_da.min_date.dt.dayofyear

Could run reproject match


In [ ]:
# Add in a CRS

dates_da_crs = dates_da.rio.write_crs(ndvi_ds_veg_only.rio.crs)
print(dates_da_crs.rio.crs)
dates_da_crs

In [ ]:
# Test plot

print(veg_gdf.crs)
print(dates_da_crs.rio.crs)

# Pick 2022 June
dates_2022_da_crs = (dates_da_crs
    .sel(year = 2022)
    .day_of_year)

# Plot with matplotlib
dates_2022_da_crs.plot(x='x', y='y', cmap=plt.colormaps['PiYG'])

## **Interactive Map of brown-down dates**

In [ ]:
simplified_veg_gdf = veg_gdf.copy()  # Make a copy to avoid modifying the original data
simplified_veg_gdf['geometry'] = veg_gdf['geometry'].simplify(tolerance=0.0001)  # Adjust the tolerance as needed

# Convert the 'STRING_COLUMN' to a string type
simplified_veg_gdf['MACROGROUP'] = simplified_veg_gdf['MACROGROUP'].astype(str)

# Create veg polygons map
veg_map = simplified_veg_gdf.hvplot(
    geo=True,
    line_color='black',
    fill_alpha=0,
    tiles='EsriImagery',
    hover_cols=['MACROGROUP']
).opts(
    width=500,
    height=800,
    show_legend=False,  # Set legend to False to remove it
    tools=['hover']
)

veg_map

# veg_map = gv.Polygons(simplified_veg_gdf, vdims=['SCIENTIFICNAME']
#     # tiles='EsriImagery'
# ).opts(
#     width=500,
#     height=800,
#     show_legend=False,  # Set legend to False to remove it
#     color=None
#     #tools=['hover']
# )

# veg_map

In [ ]:
# Convert the xarray DataArray to a GeoViews object
dates_gvds = gv.Dataset(dates_da, kdims=['x', 'y', 'year'], vdims=['day_of_year'])


# Create a GeoViews plot
dates_plot = dates_gvds.to(gv.Image).opts(cmap='viridis_r',
                                          colorbar=True,
                                          alpha=0.5,
                                          width=600,
                                          height=800,
                                          show_legend=False)


dates_veg_plot = (dates_plot  
             * veg_map).opts(tools=['hover'],
                                    width=600,
                                    height=800,
                                    show_legend=False)

# Display plot
dates_veg_plot

### **Calculate zonal means for each polygon**

In [ ]:
# Select only day of year
dates_array = dates_da['day_of_year']

# Create mask of multiple regions from shapefile
veg_mask = regionmask.mask_3D_geopandas(
    veg_gdf,
    dates_array.x,
    dates_array.y,
    drop=True,
    overlap=True,
    numbers="OBJECTID"
)

# Apply mask on dates data
dates_array = dates_array.where(veg_mask)


# Calculate means by group
veg_avg_dates_ds = (dates_array
                    .groupby("region")
                    .mean(["x","y"])
)

# Convert to dataframe
veg_avg_dates_df = (veg_avg_dates_ds.to_dataframe()
                    .apply(lambda x: x.dropna())
)

veg_avg_dates_df

There's a way to ignore NaNs in this.

In [ ]:
# Test plot

veg_avg_dates_df['day_of_year'].plot.hist()

In [ ]:
print(veg_avg_dates_df.index)

In [ ]:
# Merge average dates by vegetation patch 
# with original vegetation to retrieve geometry
veg_avg_dates_gdf = (veg_avg_dates_df
    .reset_index()
    .merge(simplified_veg_gdf, left_on='region', right_on='OBJECTID', how='left')
    .sort_values(by=['region', 'year'])

    # Convert back to gdf
    .pipe(gpd.GeoDataFrame, geometry='geometry')
    
    # Add a date
    #.assign(date_column=pd.to_datetime(veg_avg_dates_gdf['day_of_year'], format='%j', errors='coerce'))
)

# Remove rows with NaN values for plotting
veg_avg_dates_gdf = veg_avg_dates_gdf.loc[veg_avg_dates_gdf['day_of_year'].notna()]

print(type(veg_avg_dates_gdf))
veg_avg_dates_gdf



In [ ]:
# Use seaborn to create a faceted histogram
sns.set(style="whitegrid")
g = sns.FacetGrid(veg_avg_dates_gdf_nonans, col="MACROGROUP", col_wrap=2, height=4, sharex=False)

# Map the histogram to each facet
g.map(plt.hist, "day_of_year", bins=5, edgecolor='black')

# Show the plot
plt.show()

## **Chloropleth of brown-down of vegetation patches**

In [ ]:
# Set geometry
veg_avg_dates_gdf = veg_avg_dates_gdf.set_geometry('geometry')

veg_avg_dates_2019_gdf = veg_avg_dates_gdf[veg_avg_dates_gdf['year'] == 2019]


time_slider_plot = veg_avg_dates_2019_gdf.hvplot.polygons(
    geo=True, 
    c='day_of_year', 
    cmap='viridis', 
    project=True
    ).opts(
    frame_width=600, 
    frame_height=400,
    title='Vegetation brown-down in 2019',
    framewise=True,  # Enable the time slider
)

# Show the plot
time_slider_plot * gv.tile_sources.EsriImagery


In [ ]:
import pandas as pd
import geopandas as gpd
import geoviews as gv
import param
import panel as pn
gv.extension('bokeh')

In [ ]:
gdf = veg_avg_dates_gdf
print(gdf.crs)

In [ ]:
gdf = gdf.to_crs(epsg=3857)

In [ ]:
gv.Polygons(gdf
            .reset_index(),
            vdims=['day_of_year'])

In [ ]:
# gdf = gdf.reset_index()

opts = dict(width=600, height=600, tools=['hover'], colorbar=True, cmap='RdBu', 
            color='day_of_year', symmetric=True, toolbar='above',line_color='black',
            )

class PreMEI(param.Parameterized):
    year = param.Integer(default=2017, bounds=(2015, 2022))

    def chloro(self):
        return gv.Polygons(gdf[gdf['year']==self.year], vdims=['day_of_year']
)

    @param.depends('year')
    def view_chloro(self):
        return gv.DynamicMap(self.chloro).opts(**opts)

p = PreMEI()
doc = pn.Row(p.param, p.view_chloro)
doc 

In [ ]:
veg_avg_dates_gdf.hvplot.polygons(
    geo=True, 
    c='day_of_year', 
    cmap='viridis', 
    project=True)

## **Line Plots**

In [ ]:
veg_avg_dates_gdf

In [ ]:
veg_type_dates = (veg_avg_dates_gdf
                  .drop(columns='geometry')
                  .assign(year=pd.to_datetime(veg_avg_dates_gdf['year'], format='%Y'))
                  .loc[gdf['SCIENTIFICNAME'].str.contains('grassland', case=False, na=False)]  # Filter rows containing 'grassland' in the 'name' column
                  .loc[~gdf['SCIENTIFICNAME'].str.contains('shrubland', case=False, na=False)]  # Filter out rows containing 'grassland' or 'shrubland' in the 'name' column
                  .groupby(['SCIENTIFICNAME', 'year'])
                  .agg(avg_doy=('day_of_year', 'mean'))
)
veg_type_dates

In [ ]:
line_plot = veg_type_dates.hvplot.line(x='year', y='avg_doy', by='SCIENTIFICNAME', line_width=2, width=600, height=600
                            ).opts(show_legend=False,
                                   text_font_size='10pt',
                                   ylabel='average date of brown-down',
                                   title="Brown-down dates by year for grassland types in Boulder County, CO"
                            )


line_plot



## What about grasslands overall?


In [ ]:
grassland_pooled_dates = (veg_avg_dates_gdf
                  .drop(columns='geometry')
                  .assign(year=pd.to_datetime(veg_avg_dates_gdf['year'], format='%Y'))
                  .loc[gdf['SCIENTIFICNAME'].str.contains('grassland', case=False, na=False)]  # Filter rows containing 'grassland' in the 'name' column
                  .loc[~gdf['SCIENTIFICNAME'].str.contains('shrubland', case=False, na=False)]  # Filter out rows containing 'grassland' or 'shrubland' in the 'name' column
                  .groupby(['year'])
                  .agg(avg_doy=('day_of_year', 'mean'))
)
grassland_pooled_dates

In [ ]:
grassland_avg_browndown_plot = grassland_pooled_dates.hvplot.line(x='year', y='avg_doy', line_width=2, width=600, height=400
                            ).opts(
                                ylabel='average date of brown-down',
                                title="Brown-down dates by year across grasslands in Boulder County, CO"
                            )
grassland_avg_browndown_plot